In [13]:
import os
import warnings

# 커널 충돌 방지 및 경고 무시
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
warnings.filterwarnings("ignore", category=FutureWarning)

import torch
import time
from PIL import Image

# 경로 설정
YOLOV5_PATH = r'..\yolov5'
WEIGHTS_PATH = os.path.join(YOLOV5_PATH, "runs", "train", "pcb_final_run", "weights", "best.pt")
TEST_LIST_PATH = r'..\VOC_PCB\test.txt'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model = torch.hub.load(YOLOV5_PATH, 'custom', path=WEIGHTS_PATH, source='local')
model.to(DEVICE)

YOLOv5  2026-1-23 Python-3.12.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [14]:
# 테스트 데이터 리스트 확보
if os.path.exists(TEST_LIST_PATH):
    with open(TEST_LIST_PATH, 'r') as f:
        img_paths = [line.strip() for line in f.readlines() if line.strip()]
    
    durations = []
    print(f"속도 테스트 시작 (총 {len(img_paths)}장)...")

    # 속도 측정 루프
    for path in img_paths:
        if not os.path.exists(path):
            continue
            
        img = Image.open(path)
        
        # 측정 시작
        start_time = time.time()
        
        results = model(img)
        results.render()
        
        end_time = time.time()
        durations.append(end_time - start_time)

    # 결과 통계 계산 및 출력
    if len(durations) > 0:
        avg_time = sum(durations) / len(durations)
        fps = 1 / avg_time
        print("\n")
        print("="*30)
        print(f"추론 속도 평가 결과")
        print("-" * 30)
        print(f" - 평균 소요 시간: {avg_time:.4f}초")
        print(f" - 예상 FPS: {fps:.2f}")
        
        if avg_time <= 1.0:
            status = "정상"
        else:
            status = "지연"
            
        print(f" - 분석 결과 상태: {status}")
        print("="*30)
    else:
        print("측정된 데이터가 없습니다.")
else:
    print(f"오류: {TEST_LIST_PATH} 경로가 올바르지 않습니다.")

속도 테스트 시작 (총 2134장)...


추론 속도 평가 결과
------------------------------
 - 평균 소요 시간: 0.0181초
 - 예상 FPS: 55.17
 - 분석 결과 상태: 정상


In [15]:
# [추가 코드] 결과 저장 로직
save_path = r"..\qa\model_qa_results"
if not os.path.exists(save_path):
    os.makedirs(save_path)

file_name = os.path.join(save_path, "ai_speed_test_report.txt")

with open(file_name, 'w', encoding='utf-8') as f:
    f.write("=" * 30 + "\n")
    f.write("추론 속도 평가 결과\n")
    f.write("-" * 30 + "\n")
    f.write(f" - 테스트 이미지 총수: {len(img_paths)}장\n")
    f.write(f" - 평균 소요 시간: {avg_time:.4f}초\n")
    f.write(f" - 예상 FPS: {fps:.2f}\n")
    f.write(f" - 분석 결과 상태: {status}\n")
    f.write("=" * 30 + "\n")

print(f"속도 테스트 결과가 저장되었습니다: {file_name}")

속도 테스트 결과가 저장되었습니다: ..\qa\model_qa_results\ai_speed_test_report.txt
